In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rosbag
import pymap3d as pm
import numba as nb

from scipy.signal import savgol_filter
%matplotlib inline

In [2]:
def wrap_angle(angle):
    return (angle + np.pi) % (2 * np.pi) - np.pi

@nb.njit()
def to_euler(x, y, z, w):
    """Dari Coursera: Return as xyz (roll pitch yaw) Euler angles."""
    roll = np.arctan2(2 * (w * x + y * z), 1 - 2 * (x**2 + y**2))
    pitch = np.arcsin(2 * (w * y - z * x))
    yaw = np.arctan2(2 * (w * z + x * y), 1 - 2 * (y**2 + z**2))
    return np.array([roll, pitch, yaw])
# Compile the to_euler
_ = to_euler(1.5352300785980803e-15, -1.3393747145983517e-15, -0.7692164172827881, 0.638988343698562)

# Import Data

In [3]:
ls

adam_0,075.bag  adam_0,1.bag   adam_0,3.bag
adam_0,125.bag  adam_0,25.bag  cek_data.ipynb
adam_0,15.bag   adam_0,2.bag   get_data_for_simulation.ipynb


In [5]:
bag = rosbag.Bag('adam_0,15.bag')
print(bag)

path:        adam_0,15.bag
version:     2.0
duration:    27.6s
start:       Aug 22 2020 16:28:16.59 (1598088496.59)
end:         Aug 22 2020 16:28:44.18 (1598088524.18)
size:        3.4 MB
messages:    9088
compression: none [5/5 chunks]
types:       diagnostic_msgs/DiagnosticArray       [60810da900de1dd6ddd437c3503511da]
             dynamic_reconfigure/Config            [958f16a05573709014982821e6822580]
             dynamic_reconfigure/ConfigDescription [757ce9d44ba8ddd801bb30bc456f946f]
             geometry_msgs/TwistStamped            [98d34b0043a2093cf9d9345ab6eef12e]
             nav_msgs/Odometry                     [cd5e73d190d741a2f92e81eda573aca7]
             pkg_ta/Control                        [f3ff5e621674977340f826fa8303fddf]
             pkg_ta/LogArduino                     [794f6f831f1064c1b29b65087ebb232f]
             rosgraph_msgs/Log                     [acffd30cd6b6de30f120938c17c593fb]
             sensor_msgs/Imu                       [6a62c6daae103f4ff57a13

## Speed

In [ ]:
f_t = []
f_x = []
f_y = []
f_yaw = []
f_v = []
for topic, msg, _ in bag.read_messages(topics=['/odometry/filtered_map']):
    f_t.append(msg.header.stamp.to_sec())
    pos = msg.pose.pose.position
    f_x.append(pos.x)
    f_y.append(pos.y)
    q = msg.pose.pose.orientation
    euler = to_euler(q.x, q.y, q.z, q.w)
    f_yaw.append(euler[-1])
    v = msg.twist.twist.linear
    f_v.append(np.sqrt(v.x**2 + v.y**2))
f_t = np.array(f_t)
f_x = np.array(f_x)
f_y = np.array(f_y)
f_yaw = np.array(f_yaw)
f_v = np.array(f_v)